# 展示如何利用序列轉序列模型，進行問答模型的學習、測試 
## 2019.5.1. Version 1.0 by sekewei@mail.tku.edu.tw

程式目錄架構

/content/drive/My Drive/Colab Notebooks/seq2seq/ 

         chatbot2017.ipynb 模型的訓練及測試
         
         save/model/stc2017_14k/1-1_512/  模型目錄
                  x00_backup_bidir_model.tar
         
         save/training_model/stc2017_14k/  訓練所須字典，批次，語料目錄
                  x00_training_batches_64.tar
                  pairs.tar
                  voc.tar
   
         pytorch_chatbot/
                   __init__.py
                   config.py
                   evaluate.py
                   load.py
                   main.py
                   model.py
                   README.md
                   train.py


# 安裝套件

安裝pytorch環境

In [1]:
! cat /etc/issue
! python --version
! nvidia-smi

Ubuntu 18.04.1 LTS \n \l

Python 3.6.8 :: Anaconda, Inc.
Tue Apr 30 20:45:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 14%   51C    P0    59W / 250W |      0MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:07:00.0 Off |                  N/A |
| 14%   48C    P0     1W / 250W |      0MiB / 10986MiB |      0%      Default |
+------------------------------

In [2]:
#!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
#!pip3 uninstall torch
#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision
#!pip3 install torch torchvision

# 載入語料

In [3]:
#coding:utf-8
import os

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


切換工作目錄，列出工作目錄內容

In [4]:
os.chdir('/content/drive/My Drive/Colab Notebooks/seq2seq')
!ls

chatbot2017.ipynb  data  pytorch_chatbot  runtest.ipynb  save


In [1]:
corpusTxt = 'stc2017_14k.txt'
######################################
import codecs

#homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
homePath = '/home/seke/jupyter/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt

with codecs.open(filePath, "r", "utf-8") as f:
    corpus_raw = f.readlines()

corpus_raw2 = [line.strip() for line in corpus_raw]

corpus_train = {}
corpus_train['question'] = corpus_raw2[0::2]
corpus_train['answer'] = corpus_raw2[1::2]

## 利用表格檢視訓練資料 

In [2]:
import pandas as pd

pd_train = pd.DataFrame(index=range(len(corpus_train['question'])), 
                        data={'question': corpus_train['question'], 
                              'answer': corpus_train['answer']},
                       columns=['question','answer'])

pd_train

,question,answer
0,就是 太 小 了 ， 哎,所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 ~
1,作弊,我 这 不 叫 作弊 ， 称为 “ 观望 ”
2,花 姐 看 啊 快乐 大本营 。 。 ~,哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 你
3,我 呢,哈哈 ， 算 上 你 ， 她 遇上 两 个 了 ！
4,哈哈 你们 还有 骑行 活动 的 啊 。,是 呀 哈哈 ！
5,看来 你 是 一个 真 性情 之 人 ， 敢 爱 敢 恨 的 那种 ！,这 句 话 你 说 就 对了 ， 敢 爱 敢 恨
6,外国 的 天空 很 蓝 ！ 晚上 多 星星 吗 ！ ！,除了 夏威夷 晚上 看到 明亮 的 星星 ， 其他 时间 没有 留意 哦
7,"多 吃 不 长 肉 , 有 这么 神奇 的 东东 吗 ? 37",西湖 银 泰 礼券 吃 多 了 不 长 肉
8,老 南宁 已 走 远,我 的 睡眠 也 是 离 我 越来越 远 了 。 有 失眠 的 么 ？
9,打来 木 。,好 啊 。 我 号码 又 忘 了 。 ！


# 模型訓練

匯入 pytorch_chatbot 套件  
參考: [TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [1]:
import sys
import pytorch_chatbot.main as pcm
import pytorch_chatbot.evaluate as pce
import pytorch_chatbot.config as pcc

print(pcc.MAX_LENGTH)

 import train evaluate

15


## 首次訓練

In [5]:
corpusTxt = 'stc2017_14k.txt'
######################################

#training
#import main
#!python main.py -tr '../Colab Notebooks/data/corpus60w.txt' -la 1 -hi 512 -lr 0.0001 -it 400 -b 64 -p 100 -s 200 
#layer;hidden layer;learning_rate;iteration;batch_size;print/;save/

#homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
homePath = '/home/seke/jupyter/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt
cmd_line = "main.py  -tr filePath -la 1 -hi 512 -lr 0.0001 -it 6000 -b 64 -p 100 -s 200"
sys.argv = cmd_line.split()
sys.argv[2] = filePath
args = pcm.parse()
pcm.run(args)

Start loading training data ...
Saved data not found, start preparing trianing data ...
Reading lines...
Read 13990 sentence pairs
Trimmed to 9810 sentence pairs
Counting words...
Counted words: 12618
Training pairs not found, generating ...
Building encoder and decoder ...


  0%|          | 0/6000 [00:00<?, ?it/s]

Building optimizers ...
Initializing ...


  2%|▏         | 100/6000 [05:22<5:17:00,  3.22s/it]

100 1% 295.8229


...

5800 96% 1.0474


 98%|█████████▊| 5900/6000 [5:18:23<05:24,  3.24s/it]

5900 98% 1.0481


100%|██████████| 6000/6000 [5:23:44<00:00,  3.28s/it]

6000 100% 1.0454


## 後續訓練

In [2]:
corpusTxt = 'stc2017_14k.txt'
###############################
import os

homePath = '/home/seke/jupyter/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt
modelFileDir = homePath + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' 

sorted(os.listdir(modelFileDir))

['1000_backup_bidir_model.tar',
 '1200_backup_bidir_model.tar',
 '1400_backup_bidir_model.tar',
 '1600_backup_bidir_model.tar',
 '1800_backup_bidir_model.tar',
 '2000_backup_bidir_model.tar',
 '200_backup_bidir_model.tar',
 '2200_backup_bidir_model.tar',
 '2400_backup_bidir_model.tar',
 '2600_backup_bidir_model.tar',
 '2800_backup_bidir_model.tar',
 '3000_backup_bidir_model.tar',
 '3200_backup_bidir_model.tar',
 '3400_backup_bidir_model.tar',
 '3600_backup_bidir_model.tar',
 '3800_backup_bidir_model.tar',
 '4000_backup_bidir_model.tar',
 '400_backup_bidir_model.tar',
 '4200_backup_bidir_model.tar',
 '4400_backup_bidir_model.tar',
 '4600_backup_bidir_model.tar',
 '4800_backup_bidir_model.tar',
 '5000_backup_bidir_model.tar',
 '5200_backup_bidir_model.tar',
 '5400_backup_bidir_model.tar',
 '5600_backup_bidir_model.tar',
 '5800_backup_bidir_model.tar',
 '6000_backup_bidir_model.tar',
 '600_backup_bidir_model.tar',
 '800_backup_bidir_model.tar']

In [3]:
corpusTxt = 'stc2017_14k.txt'
from_iteration = 6000
###############################

#filePath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/data/corpus1218.txt'
#filePath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/data/' + seg_corpus_name + '.txt'
#modelPath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/save/model/' + seg_corpus_name + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

homePath = '/home/seke/jupyter/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt
modelPath = homePath + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' + str(from_iteration) + '_backup_bidir_model.tar'
#modelPath = homePath + 'translate2019/' + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

cmd_line = "main.py -tr filePath -l modelPath -lr 0.0001 -it 7000 -b 64 -p 100 -s 200"
sys.argv = cmd_line.split()
sys.argv[2] = filePath
sys.argv[4] = modelPath
args = pcm.parse()
pcm.run(args)


Start loading training data ...
Training pairs not found, generating ...
Building encoder and decoder ...


  0%|          | 0/1000 [00:00<?, ?it/s]

Building optimizers ...
Initializing ...


 10%|█         | 100/1000 [05:28<49:24,  3.29s/it]

6100 87% 1.0433


...

6800 97% 1.0332


 90%|█████████ | 900/1000 [49:51<05:24,  3.24s/it]

6900 98% 1.0341


100%|██████████| 1000/1000 [55:16<00:00,  3.36s/it]

7000 100% 1.0340


# 模型測試

能執行測試的條件有三

1.   有安裝torch套件
2.   有連結網路硬碟及切換工作目錄為  
        /content/drive/My Drive/Colab Notebooks/seq2seq
3.   有模型及語料字典檔    
        /content/drive/My Drive/Colab Notebooks/seq2seq/data/stc2017_14k.txt     
        /content/drive/My Drive/Colab Notebooks/seq2seq/save/training_data/stc2017_14k/400_training_batches_64.tar     
        /content/drive/My Drive/Colab Notebooks/seq2seq/save/training_data/stc2017_14k/pairs.tar    
        /content/drive/My Drive/Colab Notebooks/seq2seq/save/training_data/stc2017_14k/voc.tar  
        /content/drive/My Drive/Colab Notebooks/seq2seq/save/training_data/stc2017_14k/1-1_512/400_backup_bidir_model.tar

定義載入模型方法predictLoad()及預測答句方法predict()

In [20]:
import torch
import random
from pytorch_chatbot.train import indexesFromSentence
from pytorch_chatbot.load import loadPrepareData
from pytorch_chatbot.model import nn, EncoderRNN, LuongAttnDecoderRNN

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

#runTest(n_layers, hidden_size, reverse, modelFile, beam_size, inp, corpus):

def predictLoad(corpus, modelFile, n_layers=1, hidden_size=512):
  torch.set_grad_enabled(False)
  voc, pairs = loadPrepareData(corpus)
  embedding = nn.Embedding(voc.n_words, hidden_size)
  encoder = EncoderRNN(voc.n_words, hidden_size, embedding, n_layers)
  attn_model = 'dot'
  decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.n_words, n_layers)

  checkpoint = torch.load(modelFile)
  encoder.load_state_dict(checkpoint['en'])
  decoder.load_state_dict(checkpoint['de'])

  # train mode set to false, effect only on dropout, batchNorm
  encoder.train(False)
  decoder.train(False)

  encoder = encoder.to(device)
  decoder = decoder.to(device)
  
  return encoder, decoder, voc

def predict(encoder, decoder, voc, question, top):
  #question = '今天'
  #question = '现在 刷 朋友 圈 最大 的 快乐 就是 看 代购 们 各种 直播 '
  result_list = []

  if(top==1):
    beam_size = 1
    output_words, _ = pce.evaluate(encoder, decoder, voc, question, beam_size)
    answer = ' '.join(output_words)
    answer = answer.replace('<EOS>','')
    result_list.append(answer)
    #print(output_words)
  else:
    beam_size = top
    output_words_list = pce.evaluate(encoder, decoder, voc, question, beam_size)
    count = 0;
    for output_words, score in output_words_list:
      count = count + 1
      if(count <= top):
        output_sentence = ' '.join(output_words)
        output_sentence = output_sentence.replace('<EOS>','')
        result_list.append(output_sentence)
        #print(" {:.3f} < {}".format(score, output_sentence))
  
  return result_list

def filter(voc, question):
  words = question.split()
  result = []
  for w in words:
    if(w in voc.word2index):
      result.append(w) 
  return ' '.join(result)

設定參數，呼叫方法，載入模型，進行答句預測

In [22]:
corpusTxt = 'stc2017_14k.txt'
iteration = 6000
top = 2
###################################

n_layers = 1
hidden_size = 512

#reverse = false
# /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/voc.tar, pairs.tar, ..training_batches_64.tar

#homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
homePath = '/home/seke/jupyter/seq2seq/'
modelFile = homePath + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

en, de, voc = predictLoad(corpusTxt[:-4], modelFile, n_layers, hidden_size)


Start loading training data ...


## 批次測試

In [23]:
corpusTxt = 'stc2017_14k.txt'
######################################
import codecs

#homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
homePath = '/home/seke/jupyter/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt

with codecs.open(filePath, "r", "utf-8") as f:
    corpus_raw = f.readlines()

corpus_raw2 = [line.strip() for line in corpus_raw]

corpus_test = {}
corpus_test['question'] = corpus_raw2[0::2]
corpus_test['answer'] = corpus_raw2[1::2]

In [29]:
test_no = 10
top = 5
#######################################
corpus_test2 = {'question':corpus_test['question'][0:test_no], 'answer':corpus_test['answer'][0:test_no]}
count=0
answer_list = []
for question in corpus_test2['question']:
  count = count + 1
  question2 = filter(voc, question)
  result = predict(en, de, voc, question2, top)
  print("%3d, question: '%s'\n       filter: '%s'" % (count,question,question2))
  answer_list.append(result[0])
  for answer in result:
    print("\t'%s'" % (answer))

  1, question: '就是 太 小 了 ， 哎'
       filter: '就是 太 小 了 ， 哎'
	'所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 ~ '
	'所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 。 '
	'所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 … '
	'所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 色 了 '
	'所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 ~ ~ '
  2, question: '作弊'
       filter: '作弊'
	'我 这 不 叫 作弊 ， 称为 “ 观望 ” '
	'我 这 不 叫 作弊 ， 称为 “ 观望 地 '
	'我 这 不 叫 作弊 ， 称为 “ ” ” '
	'我 这 不 叫 作弊 ， 要不 怎么 吓 死 我 ！ '
	'我 这 不 叫 作弊 ， 要不 怎么 吓 死 你 '
  3, question: '花 姐 看 啊 快乐 大本营 。 。 ~'
       filter: '花 姐 看 啊 快乐 大本营 。 。 ~'
	'哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 你 '
	'哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 '
	'哈哈 ， 花 姐 快 睡 了 哈 。 晚安 '
	'哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 你们 '
	'哈哈 ， 花 姐 快 睡 了 。 晚安 ， 爱 你 '
  4, question: '我 呢'
       filter: '我 呢'
	'哈哈 ， 算 上 你 ， 她 遇上 两 个 了 ！ '
	'哈哈 ， 算 上 你 ， 她 遇上 两 个 了 '
	'哈哈 ， 算 上 你 ， 她 遇上 两 个 了 。 '
	'哈哈 ， 算 啊 ， 下次 再 组织 帮 你 。 '
	'哈哈 ， 算 上 你 ， 她 遇上 两 碗 了 ！ '
  5, question: '哈哈 你们 还有 骑行 活动 的 啊 。'
       filter: '哈哈 你们 还有 骑行 活动 的 啊 。'
	'是 呀 哈哈 ！ '
	'是 呀 是 呀 ！ 哈哈 … '
	'是 呀 是 呀 ！ 哈哈 '
	'是 但是 太 皮 了 。 哈哈 ！ '
	'是 呀 是 呀 哈哈 ！ '
  6, question: '看来 你 是 一个 

以表格呈現模型預測的第1答句

In [31]:
pd_test2 = pd.DataFrame(index=range(len(corpus_test2['question'])), 
                        data={'question':corpus_test2['question'], 
                              'answer':answer_list},
                       columns=['question','answer'])

pd_test2

,question,answer
0,就是 太 小 了 ， 哎,所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 ~
1,作弊,我 这 不 叫 作弊 ， 称为 “ 观望 ”
2,花 姐 看 啊 快乐 大本营 。 。 ~,哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 你
3,我 呢,哈哈 ， 算 上 你 ， 她 遇上 两 个 了 ！
4,哈哈 你们 还有 骑行 活动 的 啊 。,是 呀 哈哈 ！
5,看来 你 是 一个 真 性情 之 人 ， 敢 爱 敢 恨 的 那种 ！,哈哈 。
6,外国 的 天空 很 蓝 ！ 晚上 多 星星 吗 ！ ！,除了 夏威夷 晚上 看到 明亮 的 星星 ， 其他 时间 没有 留意 哦
7,"多 吃 不 长 肉 , 有 这么 神奇 的 东东 吗 ? 37",西湖 银 泰 礼券 吃 多 了 不 长 肉
8,老 南宁 已 走 远,像 于丹 这样 的 学者 ， 只能 使 国学 走向 空虚 ！
9,打来 木 。,好 啊 。 我 号码 又 忘 了 。 ！


## 手動測試

In [33]:
question = '终于 有 酒 喝 了 ! 希望 不会 喝 趴下'
#########################################

question2 = filter(voc, question)
result = predict(en, de, voc, question2, top)
print("%3d, question: '%s'\n       filter: '%s'" % (count,question,question2))
#answer_list.append(result[0])
for answer in result:
    print("\t'%s'" % (answer))

 10, question: '终于 有 酒 喝 了 ! 希望 不会 喝 趴下'
       filter: '终于 有 酒 喝 了 ! 希望 不会 喝 趴下'
	'嘿嘿 当然 啦 。 '
	'嘿嘿 当然 啦 '
	'嘿嘿 晚上 喝 到 晚上 '
	'嘿嘿 。 。 '
	'嘿嘿 晚上 喝 到 晚上 你 翻 就 翻 来 了 ！ '


In [34]:
corpusTxt = 'stc2017_14k.txt'
iteration = 6000
top = 2
###################################

def hand_test(voc,en,de,top,source):
  source2 = filter(voc, source)
  target = predict(en, de, voc, source2, top)
  print("source: '%s'\nfilter: '%s'" % (source,source2))
  #return target
  for answer in target:
    print("\t'%s'" % (answer))

#homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
homePath = '/home/seke/jupyter/seq2seq/'
modelFile = homePath + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

en, de, voc = predictLoad(corpusTxt[:-4], modelFile, n_layers=1, hidden_size=512)

while(True):
  source = input("Segmented Question Input: ")
  #seg_source = mecab_parse_space(source)
  hand_test(voc,en,de,top,source)

Start loading training data ...
Segmented Question Input: 喝 到 晚上
source: '喝 到 晚上'
filter: '喝 到 晚上'
	'有 没有 什么 方法 能 尽快 买 到 '
	'有 没有 什么 方法 能 尽快 买 回来 '


KeyboardInterrupt: 